In [2]:
import cudaq

import numpy as np
import scipy as sp
import scipy.stats as stats

import sympy as smp

In [3]:
def linear_indep(*mats: np.ndarray):
    vecs = [m.flatten() for m in mats]

    M = np.column_stack(vecs)
    rank = np.linalg.matrix_rank(M)
    
    return rank == len(mats)

Lie Bracket: $[A, B] = AB - BA$

$\mathfrak{g} = Lie<X, Y, Z, H, ...>$



In [11]:
# commutate
def commutator(A: np.ndarray, B: np.ndarray) -> np.ndarray:
    return A @ B - B @ A

def single_pass(op_set: list[np.ndarray]) -> tuple[list[np.ndarray], int]:
    new_ops = []
    n = len(op_set)

    # single pass to calculate commutators
    for i in range(n):
        for j in range(i+1, n):
            H = commutator(op_set[i], op_set[j])

            # continue if 0 matrix
            if np.allclose(H, 0): 
                continue

            print(f"Calculating [op[{i}], op[{j}]]")

            curr = [m.flatten() for m in (op_set + new_ops)]
            new = curr + [H.flatten()]
            rank_curr, rank_new = np.linalg.matrix_rank(curr), np.linalg.matrix_rank(new)

            if rank_new > rank_curr:
                new_ops.append(H)

    return new_ops

def compute_dla(*generators: np.ndarray, lim: int=20) -> tuple[list, int]:
    basis = list(generators)

    i = 0
    while True:
        print(f"Iteration: {i}, basis size of {len(basis)}")

        new_ops = single_pass(basis)

        if len(new_ops) == 0:
            break

        basis.extend(new_ops)
        i+=1
        
        # with np.printoptions(precision=2, suppress=True, linewidth=1000):
        for new_op in new_ops:
            print(f"Added\n{new_op}")

        if i > lim:
            print(f"iterations exceeded {lim}, DLA loop terminated")
            break

    rank = np.linalg.matrix_rank([m.flatten() for m in basis])
    return basis, rank


$
X = 
\begin{bmatrix}
0 & 0 & 1 \\
0 & 0 & 0 \\
0 & 0 & 0
\end{bmatrix},
Y = 
\begin{bmatrix}
0 & 0 & 0 \\
0 & 0 & 0 \\
1 & 0 & 0
\end{bmatrix}
$

In [5]:
X_3 = np.array([
    [0, 0, 1],
    [0, 0, 0],
    [0, 0, 0]
])

Y_3 = np.array([
    [0, 0, 0],
    [0, 0, 0],
    [1, 0, 0]
])

dla = compute_dla(X_3, Y_3)

Iteration: 0, basis size of 2
Calculating [op[0], op[1]]
Added
[[ 1  0  0]
 [ 0  0  0]
 [ 0  0 -1]]
Iteration: 1, basis size of 3
Calculating [op[0], op[1]]
Calculating [op[0], op[2]]
Calculating [op[1], op[2]]


$X, Y =$ random $U(2)$

In [6]:
X = stats.unitary_group.rvs(dim=3)
Y = stats.unitary_group.rvs(dim=3)

dla = compute_dla(X, Y)
dla

Iteration: 0, basis size of 2
Calculating [op[0], op[1]]
Added
[[ 1.03474338-0.38101406j  0.81751519-0.66406281j -0.47604749+0.06334384j]
 [ 0.92319284-0.6522201j  -0.71605988+0.53696738j  0.6256735 -0.41198068j]
 [ 0.2523711 -0.29080023j  0.33820339-0.04690978j -0.3186835 -0.15595332j]]
Iteration: 1, basis size of 3
Calculating [op[0], op[1]]
Calculating [op[0], op[2]]
Calculating [op[1], op[2]]
Added
[[ 0.79287644-0.27000446j -2.29464041-0.90640359j  1.15881389-0.13665337j]
 [ 1.1157176 +1.61391284j -0.53437905+0.16258387j -0.40765257+0.2415844j ]
 [ 0.62915673-0.99369744j  0.06076027-0.76202078j -0.25849739+0.10742059j]]
Added
[[ 1.29057711+1.57128447j -1.42752279-1.12063928j  1.16485578+0.79639306j]
 [-1.04502647-1.72403689j -1.25506562-1.51376468j  0.27328632-0.13294566j]
 [ 0.4452387 +1.17253148j  0.1003008 +0.02840492j -0.0355115 -0.05751979j]]
Iteration: 2, basis size of 5
Calculating [op[0], op[1]]
Calculating [op[0], op[2]]
Calculating [op[0], op[3]]
Calculating [op[0], op[4]

([array([[ 0.08804604-0.57246864j,  0.52585364+0.56131191j,
           0.11657583+0.2436073j ],
         [ 0.12731592+0.47847657j,  0.54023451+0.24897251j,
          -0.44895061-0.44660222j],
         [ 0.58823771-0.27084086j,  0.04234235-0.22972457j,
           0.4021547 -0.60359504j]]),
  array([[-0.13423377-0.22940426j,  0.22773626+0.87807167j,
          -0.27221345-0.17994766j],
         [ 0.11543183-0.76501983j,  0.0342587 -0.37883793j,
          -0.44306372-0.24581061j],
         [ 0.39618229+0.41691094j, -0.17878001+0.02173049j,
          -0.76843161+0.21518183j]]),
  array([[ 1.03474338-0.38101406j,  0.81751519-0.66406281j,
          -0.47604749+0.06334384j],
         [ 0.92319284-0.6522201j , -0.71605988+0.53696738j,
           0.6256735 -0.41198068j],
         [ 0.2523711 -0.29080023j,  0.33820339-0.04690978j,
          -0.3186835 -0.15595332j]]),
  array([[ 0.79287644-0.27000446j, -2.29464041-0.90640359j,
           1.15881389-0.13665337j],
         [ 1.1157176 +1.61391284j,

On 2 qubits

$X \otimes I$

$Y \otimes I$

In [12]:
I = np.array([
    [1, 0],
    [0, 1]
])

X = np.array([
    [0, 1],
    [1, 0]
])
Y = np.array([
    [0, -1j],
    [1j, 0]
])
Z = np.array([
    [1, 0],
    [0, -1]
])

dla, dim = compute_dla(np.kron(X, I), np.kron(Y, I))
dla, dim

Iteration: 0, basis size of 2
Calculating [op[0], op[1]]
Added
[[0.+2.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+2.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.-2.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.-2.j]]
Iteration: 1, basis size of 3
Calculating [op[0], op[1]]
Calculating [op[0], op[2]]
Calculating [op[1], op[2]]


([array([[0, 0, 1, 0],
         [0, 0, 0, 1],
         [1, 0, 0, 0],
         [0, 1, 0, 0]]),
  array([[0.+0.j, 0.+0.j, 0.-1.j, 0.-0.j],
         [0.+0.j, 0.+0.j, 0.-0.j, 0.-1.j],
         [0.+1.j, 0.+0.j, 0.+0.j, 0.+0.j],
         [0.+0.j, 0.+1.j, 0.+0.j, 0.+0.j]]),
  array([[0.+2.j, 0.+0.j, 0.+0.j, 0.+0.j],
         [0.+0.j, 0.+2.j, 0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j, 0.-2.j, 0.+0.j],
         [0.+0.j, 0.+0.j, 0.+0.j, 0.-2.j]])],
 np.int64(3))